# <span style="color: #158aa1;"><strong>Introduction to NaxToPy</strong></span> 

NaxToPy is a Python package developed by Idaero within the NaxTo environment that allows users to work with finite element models and results.

In the current field of aeronautical structural analysis, we deal with a massive amount of data. To facilitate the work, we rely on programs that perform iterative tasks. For instance, in a project we are currently working on, we are transitioning the analysis from Excel to Python because the 1,048,576 rows of Excel were insufficient (and it's slower).

## <span style="color: #158aa1;"><strong>How is NaxToPy installed?</strong></span>

To use NaxToPy, you need to have Python and NaxTo installed. You can install NaxToPy during the NaxTo installation process without having to run a separate command from the terminal. To check if you have already installed NaxToPy, run the command <span style="color:yellow; font-family:Consolas;">>>> pip list</span> and see if it appears in the list.

If it is not listed, you can install it like any other Python package using the command <span style="color:yellow; font-family:Consolas;">>>> pip install NaxToPy</span>

If the command fails, you may not have Python installed or it may not be added to the path.

We are using VS Code to work with jupyter notebooks. The python version is the 3.11.7 as you can see in the top right corner. As external package we are using [NaxToPy](<https://pypi.org/project/NaxToPy/>), a package that will allow as to open and load imput mesh files (.inp, .bdf, .fem) and binary result files (.op2, .odb, .h3d, .xdb, .rst). The commands and almost all the code is the same for every solver supported (Nastran, Abaqus, Optistruct and Ansys). For using this package it is necessary to have [NaxTo](<https://www.idaerosolutions.com/Home/NaxTo>) instaled. NaxTo is asuit develeped for post-prcess... bla bla bla Onece installed, using the package is as easy as import it:

## <span style="color: #158aa1;"><strong>1. Loading mesh and initialize variables</strong></span>

In [1]:
import NaxToPy as n2p  # Importing the NaxToPy library for FEM analysis.
import os              # Importing the os library for file and path handling.

# Path to the FEM input file (.dat) that contains the model information.
pathfem = r"C:\webinar_16_jan\SUBCASE_17500.dat"

# Extracting the directory from the file path.
base_path = os.path.dirname(pathfem)

# Defining the file extension for the OP2 result files.
extension = ".op2"

# Defining the number of cases and the property ID to filter elements.
num_cases = 8
property = 11


## <span style="color: #158aa1;"><strong>2. Loading load cases</strong></span>

In [2]:
# Loading the model using NaxToPy from the specified .dat file.
model = n2p.load_model(pathfem)

# Creating a list of all OP2 files in the directory that match the extension.
op2_files = [f for f in os.listdir(base_path) if f.endswith('.op2')]

# Printing the number of load cases in the model before importing results.
print(len(model.LoadCases))

# Importing results from the OP2 files into the model.
model.import_results_from_files(op2_files)

# Printing the number of load cases after importing the results.
print(len(model.LoadCases))

# Printing the load cases in the model.
print(model.LoadCases)

0
8
[N2PLoadCase(17500: '2DP'), N2PLoadCase(17501: ' PRESSURE+MECHANICAL'), N2PLoadCase(17502: ' PRESSURE+MECHANICAL+TEMPERATURE COLD'), N2PLoadCase(17503: ' PRESSURE+MECHANICAL+TEMPERATURE HOT'), N2PLoadCase(17504: ' MECHANICAL+TEMPERATURECOLD'), N2PLoadCase(17505: ' MECHANICAL+TEMPERATUREHOT'), N2PLoadCase(17506: ' PRESSURE+MECHANICAL'), N2PLoadCase(17507: ' PRESSURE+MECHANICAL+TEMPERATURE COLD')]


## <span style="color: #158aa1;"><strong>3. Extracting results</strong></span>

In [3]:
# Filtering elements in the model with the specified property ID.
elements_prop11 = [element for element in model.get_elements() if element.Prop == property]

# Getting the count of the filtered elements and printing it.
print(len(elements_prop11))

# Initializing an empty list to store results.
results2 = []

# Looping through each load case to calculate and store the maximum stress.
for lc in model.LoadCases:
    results2 = []  # Resetting the results list for each load case.
    
    # Extracting the Von Mises stress component for the current load case.
    results = lc.Results["STRESSES"].Components["VON_MISES"].get_result_ndarray()[0]
    
    # Appending the stress values for elements with the specified property.
    for e in elements_prop11:
        results2.append(results[e.InternalID])
        
    # Calculating the maximum stress for the current load case.
    stress_max = max(results2)
    
    # Printing the maximum stress for the load case.
    print("The maximum stresses for loadcase " + str(lc.ID) + " is: " + str(round(stress_max, 2)) + " MPa")

300
The maximum stresses for loadcase 17500 is: 182.75 MPa
The maximum stresses for loadcase 17501 is: 209.06 MPa
The maximum stresses for loadcase 17502 is: 352.85 MPa
The maximum stresses for loadcase 17503 is: 295.22 MPa
The maximum stresses for loadcase 17504 is: 222.3 MPa
The maximum stresses for loadcase 17505 is: 289.39 MPa
The maximum stresses for loadcase 17506 is: 165.5 MPa
The maximum stresses for loadcase 17507 is: 266.46 MPa


## <span style="color: #158aa1;"><strong>4. New derived load cases</strong></span>

In [5]:
# Creating new derived load cases by combining existing ones with a formula.
value = 7  # Number of derived load cases to create.
for i in range(value):
    loadcase = 17501 + i  # Generating load case IDs sequentially.
    
    # Creating a new derived load case using a specific formula.
    new_loadcase = model.new_derived_loadcase(
        "loadcase_t+m_" + str(i), 
        f"<LC17500:FR1>+1.5*<LC{loadcase}:FR1>"
    )
    # Printing confirmation of the creation of the new load case.
    print("The new load case has been created:", new_loadcase)

# Printing the total number of load cases after creating new ones.
print(len(model.LoadCases))

The new load case has been created: N2PLoadCase(-1: 'loadcase_t+m_0')
The new load case has been created: N2PLoadCase(-2: 'loadcase_t+m_1')
The new load case has been created: N2PLoadCase(-3: 'loadcase_t+m_2')
The new load case has been created: N2PLoadCase(-4: 'loadcase_t+m_3')
The new load case has been created: N2PLoadCase(-5: 'loadcase_t+m_4')
The new load case has been created: N2PLoadCase(-6: 'loadcase_t+m_5')
The new load case has been created: N2PLoadCase(-7: 'loadcase_t+m_6')
15


## <span style="color: #158aa1;"><strong>5. Extracting results of the new derived load cases</strong></span>

In [6]:
# Initializing variables to track the global maximum stress and corresponding load case.
global_max_stress = float('-inf')
max_lc = None

# Looping through each load case to compute derived stress components and find the maximum stress.
for lc in model.LoadCases:
    # Creating a new derived Von Mises stress component using a formula.
    lc.get_result("STRESSES").new_derived_component("von_mises", formula="sqrt(<CMPT_STRESSES:XX>^2+<CMPT_STRESSES:YY>^2-<CMPT_STRESSES:XX>*<CMPT_STRESSES:YY>+3*<CMPT_STRESSES:XY>^2)")

    # Initializing lists to store derived stress results for Z1 and Z2 sections.
    results3 = []
    results_z1 = lc.Results["STRESSES"].DerivedComponents["von_mises"].get_result_ndarray(sections=["Z1"])[0]
    results_z2 = lc.Results["STRESSES"].DerivedComponents["von_mises"].get_result_ndarray(sections=["Z2"])[0]
    
    # Combining the maximum stress from Z1 and Z2 sections for each element.
    results = [max(a, b) for a, b in zip(results_z1, results_z2)]
    
    # Filtering the results for elements with the specified property.
    for e in elements_prop11:
        results3.append(results[e.InternalID])
    
    # Finding the maximum stress for the current load case.
    stress_max = max(results3)
    
    # Printing the maximum stress for the current load case.
    print(f"The maximum stresses for load case {lc} is: {stress_max}")
    
    # Updating the global maximum stress and corresponding load case if needed.
    if stress_max > global_max_stress:
        global_max_stress = stress_max
        max_lc = lc

# Printing the overall maximum stress and the load case where it occurs.
print("The maximum stresses for loadcase " + str(max_lc.ID) + " " + max_lc.Name + " is: " + str(round(global_max_stress, 2)) + " MPa")

The maximum stresses for load case N2PLoadCase(17500: '2DP') is: 182.75305357095982
The maximum stresses for load case N2PLoadCase(17501: ' PRESSURE+MECHANICAL') is: 209.05854816466484
The maximum stresses for load case N2PLoadCase(17502: ' PRESSURE+MECHANICAL+TEMPERATURE COLD') is: 352.8504908482075
The maximum stresses for load case N2PLoadCase(17503: ' PRESSURE+MECHANICAL+TEMPERATURE HOT') is: 295.220030637563
The maximum stresses for load case N2PLoadCase(17504: ' MECHANICAL+TEMPERATURECOLD') is: 222.3045462611548
The maximum stresses for load case N2PLoadCase(17505: ' MECHANICAL+TEMPERATUREHOT') is: 289.3934465147993
The maximum stresses for load case N2PLoadCase(17506: ' PRESSURE+MECHANICAL') is: 165.49581036173666
The maximum stresses for load case N2PLoadCase(17507: ' PRESSURE+MECHANICAL+TEMPERATURE COLD') is: 266.45686374896644
The maximum stresses for load case N2PLoadCase(-1: 'loadcase_t+m_0') is: 483.57146384006967
The maximum stresses for load case N2PLoadCase(-2: 'loadcas